# Распознавание таблеток на фото

In [1]:
from pathlib import Path
import torch

from dataset_executor import download_kaggle_cv_dataset, load_cv_data
from cv_model import PillClassificationModel, PillClassificationTrainer

/media/all-c/U/Dophamine/Python/CV_classification_pill/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### В данном проекте все делается автоматически, но тестировалось на Linux (Kubuntu), дайте знать, если что-то не функционирует или работает неправильно

### Данный ноутбук (jupyter notebook) представляет из себя пайплайн, который загружает датасет, превращает его в DataLoader, затем обучает на нем модель классификации и показывает результаты ее работы на валидационной выборке. В конце следуют выводы и заметки, которые я сделал на основании работы 

In [2]:
dataset_path = download_kaggle_cv_dataset("richardradli/ogyeiv2")
print(f"Датасет находится в: {dataset_path}")

2026-01-04 21:10:04 - dataset_executor - INFO - Датасет уже существует в папке '/media/all-c/U/Dophamine/Python/CV_classification_pill/dataset/ogyeiv2' и содержит 4 файл(ов)/папок. Загрузка пропущена.


Датасет находится в: /media/all-c/U/Dophamine/Python/CV_classification_pill/dataset/ogyeiv2


In [ ]:


# Загружаем DataLoader'ы (batch_size=32 для 16GB VRAM с запасом)
train_loader, val_loader, test_loader, class_names_list = load_cv_data(
    dataset_path, 
    batch_size=32,
    image_size=224
)

print("=" * 80)
print("ИНФОРМАЦИЯ О DATALOADER'АХ")
print("=" * 80)

# Информация о каждом загрузчике
print(f"\n📊 Train DataLoader:")
print(f"   Количество батчей: {len(train_loader)}")
print(f"   Размер батча: {train_loader.batch_size}")
print(f"   Всего изображений: {len(train_loader.dataset)}")

print(f"\n📊 Validation DataLoader:")
print(f"   Количество батчей: {len(val_loader)}")
print(f"   Размер батча: {val_loader.batch_size}")
print(f"   Всего изображений: {len(val_loader.dataset)}")

print(f"\n📊 Test DataLoader:")
print(f"   Количество батчей: {len(test_loader)}")
print(f"   Размер батча: {test_loader.batch_size}")
print(f"   Всего изображений: {len(test_loader.dataset)}")

# Выводим первый батч из каждого загрузчика
print("\n" + "=" * 80)
print("ПЕРВЫЙ БАТЧ ИЗ КАЖДОГО DATALOADER'А")
print("=" * 80)

for loader_name, loader in [('Train', train_loader), 
                                ('Validation', val_loader), 
                                ('Test', test_loader)]:
    batch = next(iter(loader))
    print(f"\n🔍 {loader_name} batch:")
    print(f"   Images shape: {batch['image'].shape}")  # [batch_size, 3, 224, 224]
    print(f"   Labels shape: {batch['label'].shape}")  # [batch_size]
    print(f"   BBox shape: {batch['bbox'].shape}")     # [batch_size, 4]
    print(f"   Примеры меток: {batch['label'][:5].tolist()}")
    print(f"   Примеры классов: {batch['class_name'][:5]}")

# Список всех классов с количеством изображений
print("\n" + "=" * 80)
print("СТАТИСТИКА ПО КЛАССАМ")
print("=" * 80)
print(f"\nВсего уникальных классов в train : {len(class_names_list[0])}\n")
print(f"\nВсего уникальных классов в val: {len(class_names_list[1])}\n")
print(f"\nВсего уникальных классов в test: {len(class_names_list[2])}\n")

# Получаем статистику по каждому сплиту
for split_name, dataset in [('Train', train_loader.dataset),
                                ('Validation', val_loader.dataset),
                                ('Test', test_loader.dataset)]:
    print(f"\n📈 {split_name} dataset:")
    class_counts = dataset.get_class_counts()
    print(f"   {'─' * 50}")
    print(f"   Всего: {sum(class_counts.values())} изображений")

ИНФОРМАЦИЯ О DATALOADER'АХ

📊 Train DataLoader:
   Количество батчей: 98
   Размер батча: 32
   Всего изображений: 3136

📊 Validation DataLoader:
   Количество батчей: 21
   Размер батча: 32
   Всего изображений: 672

📊 Test DataLoader:
   Количество батчей: 21
   Размер батча: 32
   Всего изображений: 672

ПЕРВЫЙ БАТЧ ИЗ КАЖДОГО DATALOADER'А

🔍 Train batch:
   Images shape: torch.Size([32, 3, 224, 224])
   Labels shape: torch.Size([32])
   BBox shape: torch.Size([32, 4])
   Примеры меток: [47, 219, 92, 58, 87]
   Примеры классов: ['concor_10_mg_u', 'voltaren_dolo_rapid_25_mg_u', 'kalcium_magnezium_cink_s', 'diclopram_75-mg_20-mg_s', 'indastad_1_5_mg_u']

🔍 Validation batch:
   Images shape: torch.Size([32, 3, 224, 224])
   Labels shape: torch.Size([32])
   BBox shape: torch.Size([32, 4])
   Примеры меток: [0, 0, 0, 0, 1]
   Примеры классов: ['acc_long_600_mg_s', 'acc_long_600_mg_s', 'acc_long_600_mg_s', 'acc_long_600_mg_s', 'acc_long_600_mg_u']

🔍 Test batch:
   Images shape: torch.Si

#### При беглом просмотре датасета на обучение, я заметил, что там почти везде от 10 до 20 изображений. Судя по выводу, все загрузилось корректно

#### Имеются два класса, которые есть только в train датасете:

In [5]:
class_difference = set(class_names_list[0]) - (set(class_names_list[1]) or set(class_names_list[2]))
print(f"Классы, которые не встречаются в val или test: {class_difference}")

Классы, которые не встречаются в val или test: {'indastad_1_5_mg_u', 'algoflex_forte_dolo_400_mg_s'}


In [6]:
NUMBER_OF_PILL_CLASSES = len(class_names_list[0])  
BATCH_SIZE = 16
NUMBER_OF_EPOCHS = 50

# Определение устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available() and torch.cuda.get_device_name(0).startswith('AMD'):
    print(f"Используется AMD GPU: {torch.cuda.get_device_name(0)}")

# Инициализация модели
pill_classification_model = PillClassificationModel(
    number_of_pill_classes=NUMBER_OF_PILL_CLASSES,
    pretrained_backbone=True,
    dropout_rate=0.5,
    freeze_backbone_layers=False
)

# Инициализация trainer
trainer = PillClassificationTrainer(
    model=pill_classification_model,
    device=device,
    learning_rate=0.001,
    weight_decay=1e-4
)

# Цикл обучения
best_validation_accuracy = 0.0

for current_epoch in range(1, NUMBER_OF_EPOCHS + 1):
    print(f"\n{'='*60}")
    print(f"Начало эпохи {current_epoch}/{NUMBER_OF_EPOCHS}")
    print(f"{'='*60}")

    # Обучение
    train_loss, train_accuracy = trainer.train_single_epoch(
        train_dataloader=train_loader,
        epoch_number=current_epoch
    )

    # Валидация
    validation_loss, validation_accuracy = trainer.validate_model(
        validation_dataloader=val_loader,
        epoch_number=current_epoch
    )

    # Сохранение лучшей модели
    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_model_checkpoint_path = f'best_pill_model_epoch_{current_epoch}_acc_{validation_accuracy:.2f}.pth'
        trainer.save_model_checkpoint(
            checkpoint_file_path=best_model_checkpoint_path,
            epoch_number=current_epoch,
            validation_accuracy=validation_accuracy
        )
        print(f"Новая лучшая модель! Accuracy: {best_validation_accuracy:.2f}%")

2026-01-04 21:54:09,098 - cv_model - INFO - Инициализация модели для 224 классов таблеток


Используется AMD GPU: AMD Radeon RX 9070 XT


/media/all-c/U/Dophamine/Python/CV_classification_pill/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/media/all-c/U/Dophamine/Python/CV_classification_pill/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
2026-01-04 21:54:09,997 - cv_model - INFO - Модель инициализирована успешно
2026-01-04 21:54:10,190 - cv_model - INFO - Trainer инициализирован на устройстве: cuda



Начало эпохи 1/50


KeyError: 'class_index'